In [1]:
import paddlehub, paddle
import paddle.fluid as fluid
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
net = paddlehub.Module(name='face_landmark_localization')


[2020-04-22 09:32:48,851] [    INFO] - Installing face_landmark_localization module

[2020-04-22 09:32:49,605] [    INFO] - Module face_landmark_localization already installed in C:\Users\goog\.paddlehub\modules\face_landmark_localization

[2020-04-22 09:32:49,622] [    INFO] - Installing ultra_light_fast_generic_face_detector_1mb_640 module

[2020-04-22 09:32:49,629] [    INFO] - Module ultra_light_fast_generic_face_detector_1mb_640 already installed in C:\Users\goog\.paddlehub\modules\ultra_light_fast_generic_face_detector_1mb_640

In [3]:
def getEAR(data):
    dLeft = data[0]
    dRight = data[3]
    dTopLeft = data[1]
    dTopRight = data[2]
    dBottomLeft = data[5]
    dBottomRight = data[4]


    s1 = ((dTopLeft[0]-dBottomLeft[0])**2+(dTopLeft[1]-dBottomLeft[1])**2)**0.5
    s2 = ((dTopRight[0]-dBottomRight[0])**2+(dTopRight[1]
                                             -dBottomRight[1])**2)**0.5
    s3 = ((dLeft[0]-dRight[0])**2+(dLeft[1]
                                             -dRight[1])**2)**0.5
    return (s1+s2)/(2*s3)


In [4]:
def drawEye(leftE, rightE, img):
    # 眼睛加色
    mask = np.ones(img.shape, dtype=np.uint8)
    
    ellipsisL = cv2.fitEllipseDirect(leftE)
    ellipsisR = cv2.fitEllipseDirect(rightE)
    valL = ((ellipsisL[0][0]+3, ellipsisL[0][1]),
           (ellipsisL[1][0]+8, ellipsisL[1][1]+12),
            ellipsisL[2]+3)
    valR = ((ellipsisR[0][0]+8, ellipsisR[0][1]),
           (ellipsisR[1][0]+8, ellipsisR[1][1]+10),
            ellipsisR[2]+3)
    cv2.ellipse(mask, valL, (0, 255, 255), -1, 
                lineType=cv2.LINE_8)
    cv2.ellipse(mask, valR, (0, 255, 255), -1, 
                lineType=cv2.LINE_8)
    cv2.addWeighted(img, 0.9, mask, 0.8, 0, img)
    return img

In [15]:
cap = cv2.VideoCapture(0)
next_frame = 0
L_EARLess0_3Count = 0
blinkCount = 0
R_EARLess0_3Count = 0
always_eye = 0
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    # 图片转换
    # cv2.imshow("goog", frame)
    re = net.keypoint_detection(images=[frame], use_gpu=False)
    detect_frame = frame.copy()
    try:
        # print(re)
        for face in re[0]['data']:  # 脸
        # print(result)
        #     for data in face[36:48]:  # 脸数据 / 眼睛数据
                # cv2.circle(detect_frame, (int(data[0]), int(data[1])), radius=2, 
                #            thickness=2, color=(255, 0, 255))
                # 描绘轮廓
                leftEye = np.asarray([[int(i[0]), int(i[1])] 
                                      for i in face[36:42]])
                rightEye = np.asarray([[int(i[0]), int(i[1])] 
                                      for i in face[42:48]])
                # leftEyeHull = cv2.convexHull(leftEye)
                # rightEyeHull = cv2.convexHull(leftEye)
                
                # cv2.drawContours(detect_frame, [leftEye],
                #                  -1, (0, 255, 255), 1)  # 左眼轮廓
                # cv2.drawContours(detect_frame, [rightEye],
                #                      -1, (0, 255, 255), 1)  # 右眼轮廓
                
                
                # 计算 EAR值 
                #  左眼EAR
                
                # if next_frame % 3 == 0:
                #     next_frame=0
                leftEAR = getEAR(face[36:42])
                rightEAR = getEAR(face[42:48])
                if leftEAR<0.3:
                    L_EARLess0_3Count+=1
                else:
                    if L_EARLess0_3Count > 7:
                            always_eye = 1
                    if L_EARLess0_3Count>=3:
                        L_EARLess0_3Count=0
                        detect_frame = drawEye(leftEye, rightEye, 
                                               detect_frame)
                        blinkCount+=1
                    
                cv2.putText(detect_frame, "L-Blink:"+str(blinkCount), (10, 100), 
                                cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.6, 
                                (0, 255, 255), 1)
                # next_frame+=1
                cv2.putText(detect_frame, "L-EAR:"+str(leftEAR), (10, 40), 
                                cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.6, 
                                (255, 255, 255), 1)
                cv2.putText(detect_frame, "R-EAR:"+str(leftEAR), (10, 80), 
                            cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.6, 
                            (255, 255, 255), 1)
    except Exception as e:
        print("no detect")
    
    # 持续放大招
    if always_eye == 1:
        detect_frame = drawEye(leftEye, rightEye, 
                                               detect_frame)
    cv2.imshow('goog_detect', detect_frame)
        
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
# cv2.imwrite("goog.jpg", frame)
cap.release()
cv2.destroyAllWindows()


In [10]:
import os
import time
import numpy as np
img = cv2.imread("goog.jpg")
start = time.time()
re = net.keypoint_detection(images=[img], use_gpu=False,
                   visualization=False)
result = re[0]['data'][0]
leftEAR = getEAR(result[36:42])
cv2.putText(img, "R-EAR:"+str(leftEAR), (10, 40), 
                                cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.5, 
                                (255, 255, 255), 1)
cv2.putText(img, "R-EAR:"+str(leftEAR), (10, 80), 
                                cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.5, 
                                (255, 255, 255), 1)
# print(result[36:42])
a = np.asarray([[int(i[0]), int(i[1])] for i in result[36:42]])
# a = a.reshape((-1,  1, 2))
# print(a)
# leftEyeHull = cv2.convexHull(points=a)
# print(leftEyeHull)
img1 = img.copy()
mask = np.ones(img1.shape, dtype=np.uint8)

ellipsis = cv2.fitEllipseDirect(a)
val = ((ellipsis[0][0]-3, ellipsis[0][1]),
       (ellipsis[1][0]+8, ellipsis[1][1]+8), ellipsis[2]+3)
# print(val)
cv2.ellipse(mask, val, (0, 255, 255), -1, lineType=cv2.LINE_8)
# cv2.drawContours(img, [a], -1, (0, 0, 255), 3, 
#                  lineType=cv2.LINE_8,
#                  offset=(0, 0))  # 左眼轮廓
print(mask.dtype)
cv2.addWeighted(img, 0.9, mask, 0.8, 0, mask)
# cv2.rectangle(img, (int(result['left']), int(result['top'])),
#                       (int(result['right']), int(result['bottom'])), (0, 0, 255), 2)
end = time.time()
cv2.imshow("goog", mask)
print("time:%.4f" % (end-start))
# print(re[0]['data'][0])
cv2.waitKey(0)

uint8

time:0.1297

-1